In [1]:
import os
import glob
import numpy as np
import pathlib
import SimpleITK as sitk
import scipy.spatial.distance as distance
from dask import delayed,diagnostics,multiprocessing,compute
%run functions.ipynb

In [16]:
XY_RES = 0.868
Z_RES = 50

exp_name = 'MG50_rhs/'
input_image_path =  '/nas1/Data_Mythreya/MotorCortexProject/V0/0_Inputs/Images/Rabies/NeuN/' + exp_name
input_contours_path = '/nas1/Data_Mythreya/MotorCortexProject/V0/0_Inputs/Contours/Rabies/' + exp_name
#output_path = '/nas1/Data_Mythreya/MotorCortexProject/V0/0_Inputs/Landmarks/Rabies/NeuN/' + exp_name

output_path = '/rall/rabies_neun_counts/' + exp_name
pathlib.Path(output_path).mkdir(exist_ok= True)
sec_num_start = 36
sec_num_end = 40
#sec_num = 45


In [17]:
@delayed
def count_cells(i):
    if os.path.exists(output_path+'S_{}/landmarks.landmarkAscii'.format(i)):
        landmarks = getLandmarkCoords(output_path+'S_{}/landmarks.landmarkAscii'.format(i))
        if len(landmarks) > 0:
            # count already exists... so skip this section
            return
    print(i)
    
    try:
        pathlib.Path(output_path+'S_{}/'.format(i)).mkdir(exist_ok= True)
        # mask the image so that only cortex is used for counting
        im = sitk.ReadImage(input_image_path+'S{}_01.tif'.format(i))
        spatial_graph_file = input_contours_path + 'S{:03d}_ct.am'.format(i)
        pia,wm = getSectionPiaWMPoints(spatial_graph_file)
        pia_bin = convertContourCoordsIntoBinaryImage(pia,im,resample_res=0.1)
        wm_bin = convertContourCoordsIntoBinaryImage(wm,im,resample_res=0.1)

        pia_only = sitk.Mask(im,pia_bin)
        cortex_im = sitk.Mask(pia_only,sitk.Not(wm_bin))
        sitk.WriteImage(cortex_im,output_path+'S_{}/S_{}.tif'.format(i,i))
        with open(output_path+'S_{}'.format(i)+'/arguments_{}.txt'.format(i),'w') as f:
            f.write('<command> CMD_COUNT \n')
            f.write('<ifilename> {}S_{}/S_{}.tif\n'.format(output_path,i,i))
            f.write('<ofilename>    {}\n'.format(output_path+'S_{}/'.format(i)))
            f.write('<cfilename>    {}/CellClusterList.csv\n'.format(output_path+'section_{}/'.format(i)))
            f.write('<scantype> 1\n')
            f.write('<xysampling> 0.868\n')
            f.write('<zsampling> 1\n')
            f.write('<bricksize>  300 m\n')
            f.write('<somaradius> 4\n')
            f.close()
        os.system('/home/mythreya//projects/CellCount/source_code/bugfix/Version20120306_ColocV2/Version20120306_ColocV2_Copy/build/CellCount \
        {} > {}'.format(output_path+'S_{}'.format(i)+'/arguments_{}.txt'.format(i),output_path+'S_{}'.format(i)+'/screen_outputs{}.txt'.format(i)))
    
    except RuntimeError:
        print('Image file not found')

In [18]:
fun_list = []
for i in range(sec_num_start,sec_num_end+1):
    fun_list.append(count_cells(i))
    if (len(fun_list)%5)==0:
        compute(fun_list)
        fun_list = []


36
383739
40


